In [2]:
import sys
import requests
import json

In [5]:
from bs4 import BeautifulSoup as bs
from contextlib import closing
import urllib
import re

# BeautifulSoup

<html>
    <head>
        <title>The Dormouse's story</title>
    </head>
    <body>
        <p class="title" name="dromouse">
            <b>
                The Dormouse's story
            </b>
        </p>
        <p class="story">
        Once upon a time there were three little sisters; and their names were
            <a href="http://example.com/elsie" class="sister" id="link1">
            <!-- Elsie -->
            </a>,
            <a href="http://example.com/lacie" class="sister" id="link2">
            Lacie
            </a> 
            and
            <a href="http://example.com/tillie" class="sister" id="link3">
            Tillie
            </a>;
            and they lived at the bottom of a well.
        </p>
        <p class="story">...</p>
    </body>
</html>

[Internal Node] head
[Internal Node] title
[Leaf Node] The Dormouse's title
[Internal Node] body
[Internal Node] p
[Internal Node] b
[Leaf Node] The Dormouse's story
[Internal Node] p
[Leaf Node] Once upon a time there were three little sisters; and their names were

[Internal Node] a
[Leaf Node]  Elsie 
[Leaf Node] ,

[Internal Node] a
[Leaf Node] Lacie
[Leaf Node]  and

[Internal Node] a
[Leaf Node] Tillie
[Leaf Node] ;
and they lived at the bottom of a well.
[Internal Node] p
[Leaf Node] ...

In [6]:
html = """
<html><head><title>The Dormouse's title</title></head>
<body>
<p class="title" name="dromouse"><b>The Dormouse's story</b></p>
<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1"><!-- Elsie --></a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>
<p class="story">...</p>
</body>
</html>
"""

In [7]:
soup = bs(html,'html.parser')

In [8]:
print soup.prettify()

<html>
 <head>
  <title>
   The Dormouse's title
  </title>
 </head>
 <body>
  <p class="title" name="dromouse">
   <b>
    The Dormouse's story
   </b>
  </p>
  <p class="story">
   Once upon a time there were three little sisters; and their names were
   <a class="sister" href="http://example.com/elsie" id="link1">
    <!-- Elsie -->
   </a>
   ,
   <a class="sister" href="http://example.com/lacie" id="link2">
    Lacie
   </a>
   and
   <a class="sister" href="http://example.com/tillie" id="link3">
    Tillie
   </a>
   ;
and they lived at the bottom of a well.
  </p>
  <p class="story">
   ...
  </p>
 </body>
</html>



In [9]:
soup.text

u"\nThe Dormouse's title\n\nThe Dormouse's story\nOnce upon a time there were three little sisters; and their names were\n,\nLacie and\nTillie;\nand they lived at the bottom of a well.\n...\n\n\n"

In [10]:
all_a = soup.find_all("a")
print all_a
print type(all_a)


[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]
<class 'bs4.element.ResultSet'>


In [9]:
all_a[0]["href"]

u'http://example.com/elsie'

In [11]:
soup.p

<p class="title" name="dromouse"><b>The Dormouse's story</b></p>

In [11]:
# Depth First Search
def recursiveChildren(x):
    if "childGenerator" in dir(x):
        for child in x.childGenerator():
            name = getattr(child, "name", None)
            if name is not None:
                print "[Internal Node]",child.name
            recursiveChildren(child)
    else:
        if not x.isspace(): #Just to avoid printing "\n" parsed from document.
            print "[Leaf Node]",x

In [12]:
soup.text

u"\nThe Dormouse's title\n\nThe Dormouse's story\nOnce upon a time there were three little sisters; and their names were\n,\nLacie and\nTillie;\nand they lived at the bottom of a well.\n...\n\n\n"

In [13]:
for child in soup.childGenerator():
    recursiveChildren(child)

[Internal Node] head
[Internal Node] title
[Leaf Node] The Dormouse's title
[Internal Node] body
[Internal Node] p
[Internal Node] b
[Leaf Node] The Dormouse's story
[Internal Node] p
[Leaf Node] Once upon a time there were three little sisters; and their names were

[Internal Node] a
[Leaf Node]  Elsie 
[Leaf Node] ,

[Internal Node] a
[Leaf Node] Lacie
[Leaf Node]  and

[Internal Node] a
[Leaf Node] Tillie
[Leaf Node] ;
and they lived at the bottom of a well.
[Internal Node] p
[Leaf Node] ...


In [14]:
print soup.title

<title>The Dormouse's title</title>


In [15]:
print soup.head

<head><title>The Dormouse's title</title></head>


In [12]:
print soup.a

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>


In [17]:
print soup.p

<p class="title" name="dromouse"><b>The Dormouse's story</b></p>


In [18]:
#NavigableString
print soup.p.string
print type(soup.p.string)
print type(soup.p.string)

The Dormouse's story
<class 'bs4.element.NavigableString'>
<class 'bs4.element.NavigableString'>


In [ ]:
#（3）BeautifulSoup

print soup.name

In [19]:
#Comment
print soup.a
print soup.a.string
print type(soup.a.string)

<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>
 Elsie 
<class 'bs4.element.Comment'>


In [20]:
soup.find_all("a")

[<a class="sister" href="http://example.com/elsie" id="link1"><!-- Elsie --></a>,
 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>,
 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>]

In [ ]:
def get_user_id(user_profile, user_ids):
    url = user_profile
    with closing(urllib.urlopen(url)) as page:
        for line in page:
            if "steamid" in line:
                try:
                    user_id = re.search("\"steamid\":\"(\d+)\"", line).group(1)
                    print user_id + ' ' + user_profile
                    if user_id != None:
                        user_ids.append(user_id)
                        break
                except:
                    continue


In [ ]:
def get_online_users(member_list_no, user_ids):
    url = 'https://steamcommunity.com/games/steam/members?p=' + str(member_list_no)
    resp = requests.get(url)
    soup = bs(resp.text, 'html.parser')
    # print(soup.prettify())
    # search profile of users who are online/in-game
    all_users = soup.find_all("div", \
                              onclick = re.compile("top\.location\.href='https:\/\/steamcommunity\.com\/id\/(\w+)'"), \
                              class_ = re.compile("online|in-game"))
# get user names
    for user in all_users:
        user_profile = user.div.div.div.a['href'].encode("ascii")
# print user_profile
        get_user_id(user_profile, user_ids)

In [ ]:
member_list_page_no = 1
user_ids = []
for idx in range(1, member_list_page_no + 1):
    print "Member List " + str(idx)
    get_online_users(idx, user_ids)
    print "Total online users found:"
    print len(user_ids)

# API

In [ ]:
app_id=70
key='7BA13DD5D9CD290F7450FA29F09D0F39'
#http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=7BA13DD5D9CD290F7450FA29F09D0F39&steamids=76561197960435530
url = 'http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key=' + key + '&steamids=76561197971155734'

header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4,zh-TW;q=0.2'
}
resp = requests.get(url, header)
obj = resp.json()
print obj

### GET APP DETAIL

In [1]:
app_id=70
url = 'http://store.steampowered.com/api/appdetails?appids=%s' % app_id
header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4,zh-TW;q=0.2'
}
resp = requests.get(url, header)
obj = resp.json()
print obj


NameError: name 'requests' is not defined

## GET Profile Name

In [3]:
import re
profileURL = 'http://steamcommunity.com/profiles/%s' % '76561197960265738'
respProfile = requests.get(profileURL, header)
regxObj = re.compile('"personaname":"(.+?)"', re.IGNORECASE | re.UNICODE | re.DOTALL)
match = regxObj.search(respProfile.text)
print match

ConnectionError: HTTPConnectionPool(host='steamcommunity.com', port=80): Max retries exceeded with url: /profiles/76561197960265738?Accept-Language=en-US%2Cen%3Bq%3D0.8%2Czh-CN%3Bq%3D0.6%2Czh%3Bq%3D0.4%2Czh-TW%3Bq%3D0.2&Accept-Encoding=gzip%2C+deflate%2C+sdch&Accept=text%2Fhtml%2Capplication%2Fxhtml%2Bxml%2Capplication%2Fxml%3Bq%3D0.9%2Cimage%2Fwebp%2C%2A%2F%2A%3Bq%3D0.8&User-Agent=Mozilla%2F5.0+%28Macintosh%3B+Intel+Mac+OS+X+10_11_6%29+AppleWebKit%2F537.36+%28KHTML%2C+like+Gecko%29+Chrome%2F57.0.2987.133+Safari%2F537.36 (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x1062f8990>: Failed to establish a new connection: [Errno 60] Operation timed out',))

In [25]:
import re
profileURL = 'http://steamcommunity.com/profiles/%s' % '76561197960265738'
respProfile = requests.get(profileURL, header)
regxObj = re.compile('"personaname":"(.+?)"', re.IGNORECASE | re.UNICODE | re.DOTALL)
#print respProfile.text

In [26]:
print respProfile.text

<!DOCTYPE html>
<html class=" responsive" lang="en">
<head>
	<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
			<meta name="viewport" content="width=device-width,initial-scale=1">
		<meta name="theme-color" content="#171a21">
		<title>Steam Community :: Gregori</title>
	<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon">

	<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/motiva_sans.css?v=Sd0odMs2NjL1" rel="stylesheet" type="text/css" >
<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/buttons.css?v=FMXZx9fv9yp_" rel="stylesheet" type="text/css" >
<link href="http://steamcommunity-a.akamaihd.net/public/shared/css/shared_global.css?v=ezCZ54xw7bxo" rel="stylesheet" type="text/css" >
<link href="http://steamcommunity-a.akamaihd.net/public/css/globalv2.css?v=3X1Hz39cVBqV" rel="stylesheet" type="text/css" >
<link href="http://steamcommunity-a.akamaihd.net/public/css/skin_1/modalContent.css?v=.vXD4gU69vdZP" rel="stylesheet

In [27]:
regxObj = re.compile('"personaname":"(.+?)"', re.IGNORECASE | re.UNICODE | re.DOTALL)

In [16]:
if match:
    obj = match.group(1)
else:
    obj = []
print obj    

Gregori


In [ ]:
regxObj = re.compile('"personaname":"(.+?)"', re.IGNORECASE | re.UNICODE | re.DOTALL)
match = regxObj.search(respProfile.text)
save

In [ ]:
gurl = 'http://steamcommunity.com/profiles/%s/games/?tab=all' % '76561197960265738'
respGames = requests.get(profileURL, header)
regxObj = re.compile('var rgGames = (.+?);\s*?var', re.IGNORECASE | re.UNICODE | re.DOTALL)
match = regxObj.search(respGames.text)
print match
if match:
    obj = json.loads(match.group(1))
else:
    obj = []
print obj

for item in obj:
    print item

In [ ]:
print respGames.text

In [ ]:
proxies = { "http": "http://10.10.1.10:3128", "https": "http://10.10.1.10:1080", }   
app_id=70
url = 'http://store.steampowered.com/api/appdetails?appids=%s' % app_id
header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4,zh-TW;q=0.2'
}
resp = requests.get(url, header)
obj = resp.json()
print obj

In [ ]:
patterns = {
        'detail': {
            #'steamID': False,
            'name': '"personaname":"(.+?)"',
            'real_name': '<div class="header_real_name ellipsis">[^<>]*?<bdi>([^<>]+?)</bdi>',
            'location': '<img class="profile_flag" src="[^>]*?">([^<>]+?)</div>',
            'level': '<span class="friendPlayerLevelNum">(\d+?)</span>',
            'since': 'data-community-tooltip="Years of Service<br>[^<>]*?Member since ([^<>]+?).">',
            'customURL': '"url":"([^<>]+?)",',
            'games_url': '<a href="([^<>]+?)">\s*?<span class="count_link_label">Games</span>',
            'friends_url': '<a href="([^<>]+?)">\s*?<span class="count_link_label">Friends</span>'
        },
        'games': {
            'gameList': 'var rgGames = (.+?);\s*?var',
            'error': 'made too many requests recently. Please wait and try your request again later.'
        },
        'friends': {
            'friendList': '<a class="friendBlockLinkOverlay" href="([^<>]+?)"></a>.+?<div class="friendBlockContent">([^<>]+?)<br>'
            # Wierd, there is one time I didn't see the pattern above, but only the pattern below
            # The pattern above doesn't exist in my own friends page! But in other people's friends pages
            #'friendList': '<a class="friendBlockLinkOverlay" href="([^<>]+?)"></a><div class=.+?>.+?</div><div>([^<>]+?)<br />'
        },
        'games_url': '<a href="([^<>]+?)">\s*?<span class="count_link_label">Games</span>',
        'friends_url': '<a href="([^<>]+?)">\s*?<span class="count_link_label">Friends</span>'
    }

profileURL = 'http://steamcommunity.com/profiles/%s' % 76561197960265738


class ContentRetriever(object):

    def _re_search(self, pattern, html):
        regxObj = re.compile(pattern, re.IGNORECASE | re.UNICODE | re.DOTALL)
        match = regxObj.search(html)
        return match

    def _re_finditer(self, pattern, html):
        regxObj = re.compile(pattern, re.IGNORECASE | re.UNICODE | re.DOTALL)
        return regxObj.finditer(html)

#here is your code#


In [ ]:
proxy = {
  "https": "https://23.253.83.226:80"
}
app_id=70
url = 'http://store.steampowered.com/api/appdetails?appids=%s' % app_id
header = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
        'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding':'gzip, deflate, sdch',
        'Accept-Language':'en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4,zh-TW;q=0.2'
}
resp = requests.get(url, header, timeout=10, proxies=proxy)
obj = resp.json()
print obj
